In [1]:
import os
import pandas as pd
import csv
import numpy as np


In [2]:
#### datapath for lenovo laptop

master_file_path = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Master File\\CompanyMaster.csv"
directory = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Data Files\\"  # Replace with the path to your directory
combined_directory = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Delivery Price Combined Files\\"  # Replace with the path to your directory

input_path =  "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Input File\\"
output_path = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Output File\\"
column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']

non_fno_csv = output_path + '0_non_fno.csv'
fno_csv = output_path + 'fno.csv'
sme_csv = output_path +'sme_csv.csv'

avgs_file = input_path + 'ShortListed Stocks.xlsx'

In [3]:
def replace_nse_code(row):
    if pd.isna(row['NSE_Code']) or row['NSE_Code'] == "":
        return row['BSE_Code']
    else:
        return row['NSE_Code']
    
def add_nse_prefix(df, column_name):
    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    df = df.copy()

    # Add "NSE:" prefix to the specified column
    df.loc[:, column_name] = "NSE:" + df[column_name].astype(str)

    return df

In [4]:
no_days = 2

file_names = sorted([file for file in os.listdir(combined_directory) if file.lower().endswith(("CSV","csv"))
                        and file.startswith("combined")], reverse=True)[:no_days]

print(file_names)

count = 1

for file_name in file_names:
    
    file_path = os.path.join(combined_directory,file_name)
    csv_file_df = pd.read_csv(file_path)
    
    if count == 1:
        combined_df = csv_file_df
    else:
        combined_df = pd.concat([combined_df, csv_file_df], ignore_index=True)
    
    count += 1
    #print(count)

combined_df['BSE_Code'] = combined_df['BSE_Code'].fillna(0)
combined_df['BSE_Code'] = combined_df['BSE_Code'].astype(int)
combined_df['NSE_Code'] = combined_df['NSE_Code'].fillna("")
combined_df['Company Name'] = combined_df['Company Name'].fillna("")
combined_df['NSE_Code'] = combined_df.apply(replace_nse_code, axis=1)

report_date = file_names[0]
report_date = report_date[9:15]
print(report_date)


['combined_231226.csv', 'combined_231222.csv']
231226


In [5]:
#combined_df.info()


In [6]:
blank_co = combined_df[combined_df['Company Name'] == ""]
#blank_co

In [7]:
dely_value = 50000000


# Assuming your DataFrame is named combined_df
# Sort the DataFrame by Date
combined_df.sort_values(by='Date', inplace=True)

# Create a new DataFrame for the later date and the previous date
later_date = combined_df[combined_df['Date'] == combined_df['Date'].max()]
previous_date = combined_df[combined_df['Date'] == combined_df['Date'].min()]

# Merge the two DataFrames on Company Name, NSE Code, and BSE Code
merged_df = pd.merge(later_date, previous_date, on=['Company Name', 'NSE_Code', 'BSE_Code'], how='outer', 
                     suffixes=('_later', '_previous'))

# Exclude rows where 'Total_Delivery_later' is 0
merged_df = merged_df[merged_df['Total_Delivery_later'] != 0]

# Exclude rows where delivery value today is less than 10000000
merged_df = merged_df[(merged_df['Total_Delivery_later'] * merged_df['Price_later']) >= dely_value]

# Select the desired columns
result_df = merged_df[['Company Name', 'NSE_Code', 'Total_Delivery_later', 'Total_Delivery_previous', 'Price_later', 'Price_previous']].copy()

column_names = ['Company Name','ScripCode','Dely','PrevDel','Price','PrevPrice']

# Rename columns
result_df.columns = column_names

result_df['Dely'] = result_df['Dely'].astype(int)
result_df['PrevDel'] = result_df['PrevDel'].fillna(0)
result_df['PrevDel'] = result_df['PrevDel'].astype(int)


# Calculate the ratio of Total_Delivery_later to Total_Delivery_previous
result_df['DelyRatio'] = result_df['Dely'] / result_df['PrevDel']

# Sort the result DataFrame in descending order based on the ratio
result_df.sort_values(by='DelyRatio', ascending=False, inplace=True)

# Fill NaN values with 0 for cases where there is no previous date
result_df.fillna(0, inplace=True)

# Add a new column 'PriceChange' indicating whether the price went up or down
result_df['PriceChange'] = np.where(result_df['Price'] > result_df['PrevPrice'], 'Up', np.where(result_df['Price'] < result_df['PrevPrice'], 'Down', 'No Change'))


filtered_data = result_df[result_df['DelyRatio'] >= 3]

result_df = filtered_data
# Drop rows where "Company Name" is empty
# Replace empty strings with NaN
result_df['Company Name'].replace('', np.nan, inplace=True)
result_df.dropna(subset=['Company Name'], inplace=True)

result_df.reset_index(drop=True, inplace=True)


# Format the alignment of the 'Company_Name' column to left-align
#result_df.style.set_properties(subset=['Company_Name'], **{'text-align': 'left'})
result_df.style.set_properties(subset=['Company Name'], **{'text-align': 'left'})

formatted_columns = ['Company Name', 'ScripCode','Dely','PrevDel', 'Price', 'PrevPrice','DelyRatio', 'PriceChange']
formatted_data = result_df[formatted_columns].copy()

formatted_data['Dely'] = formatted_data['Dely'].apply(lambda x:f"{int(x):,}")
formatted_data['PrevDel'] = formatted_data['PrevDel'].apply(lambda x:f"{int(x):,}")
formatted_data['DelyRatio'] = formatted_data['DelyRatio'].apply(lambda x:f"{x:.2f}")
formatted_data['Price'] = formatted_data['Price'].apply(lambda x: f"{round(x, 2):,.2f}")
formatted_data['PrevPrice'] = formatted_data['PrevPrice'].apply(lambda x: f"{round(x, 2):,.2f}")

styled_result = formatted_data.style.set_properties(subset=['Company Name', 'ScripCode'], **{'text-align': 'left'})

# Display the result in a formatted way
#print(styled_result.to_string(index=False))
#print("Delivery comparision over 2 days ", report_date, " value over ", dely_value/100000)
#print(" ")
#print(" ")
#display(styled_result)


In [8]:
columns_to_select = ['Company Name', 'ScripCode', 'Dely', 'PrevDel', 'Price', 'PrevPrice', 'DelyRatio', 'PriceChange']
col_length = [30, 12, 12, 12, 12, 12, 8, 6]
formatted_columns = [col + ' ' * (length - len(col)) for col, length in zip(columns_to_select, col_length)]

def print_formatted_rows(df, columns_to_select):
    for i, (_, row) in enumerate(df.iterrows(), 1):
        p_coname = row[columns_to_select[0]].ljust(35)
        p_cocode = str(row[columns_to_select[1]]).ljust(12)  # Convert to string before formatting
        p_dely = f"{row[columns_to_select[2]]:,.0f}".rjust(12)
        p_prevdely = f"{row[columns_to_select[3]]:,.0f}".rjust(12)  # Correct the typo here
        p_price = f"{row[columns_to_select[4]]:,.2f}".rjust(12)
        p_prevprice = f"{row[columns_to_select[5]]:,.2f}".rjust(12)  # Correct the typo here
        p_delyratio = f"{row[columns_to_select[6]]:,.2f}".rjust(8)  # Correct the typo here
        p_direction = str(row[columns_to_select[7]]).ljust(6)  # Convert to string before formatting

        print(f"{p_coname}{p_cocode}{p_dely}{p_prevdely}{p_price}{p_prevprice}  {p_delyratio}    {p_direction}")

        if i % 5 == 0:
            print()

# Example usage:
# print_formatted_rows(your_dataframe, columns_to_select)


In [9]:
title = " " * 30 + "Delivery comparision over 2 days, value over 500, ratio over 3x  " + str(report_date) 
title_1 = " " * 30 + "_"*70

header = "Company Name" + " "*23 +"Symbol" + " " *12 + "Dely" + " "*8 + "PrevDely" + " "*4 + "Price" + " " * 4
header = header + "PrevPrice" + " "*4 + "Ratio" + " "*3 + "Change"
header_2 = "="*30 + " "*5 + "="*80


print(title)
print(title_1)
print(" ")
print(" ")
print(header)
print(header_2)
print(" ")


print_formatted_rows(result_df,columns_to_select)

                              Delivery comparision over 2 days, value over 500, ratio over 3x  231226
                              ______________________________________________________________________
 
 
Company Name                       Symbol            Dely        PrevDely    Price    PrevPrice    Ratio   Change
==============================     ================================================================================
 
Motisons Jewellers Limited         MOTISONS      12,711,551           0      103.55        0.00       inf    Up    
Suraj Estate Developers Limite     SURAJEST       2,432,857           0      334.50        0.00       inf    Up    
Muthoot Microfin Limited           MUTHOOTMF      5,052,794           0      266.15        0.00       inf    Up    
Simbhaoli Sugars Ltd.              SIMBHALS       2,539,160      25,270       33.60       28.05    100.48    Up    
JSW Holdings Ltd.                  JSWHL            106,282       1,338    5,285.55    5,052.20  

In [10]:
volume_3x = output_path + '_' + report_date +  "_0_volume_3x_wl" + ".csv"
volume_df_wl = result_df[['ScripCode']]

volume_df_nse_wl = add_nse_prefix(volume_df_wl,'ScripCode')

volume_df_nse_wl.to_csv(volume_3x, index=False)

In [11]:
numeric_code_df = result_df[result_df['ScripCode'].apply(lambda x: isinstance(x, (int, float)))]

print(numeric_code_df[['ScripCode', 'Company Name', 'Price', 'Dely', 'DelyRatio']])


   ScripCode                    Company Name   Price     Dely  DelyRatio
9     514010            Himachal Fibres Ltd.   29.67  4026974  15.112865
16    531859  Oriental Rail Infrastructure L  235.25   822312  10.966646
53    505710      Grauer & Weil (India) Ltd.  132.90   420592   4.940295
88    531112      Balu Forge Industries Ltd.  250.95   255075   3.389025
